In [25]:
from pprint import pprint
from copy import deepcopy
from collections import namedtuple

import pandas as pd
import numpy as np
import random

df = pd.DataFrame().from_dict({
  1:['#facade', '#639'],
  2:['#639', '#facade']
})

df.head()

,1,2
0,#facade,#639
1,#639,#facade


In [26]:
df.style.applymap(
  lambda x: f'color:{x}; background-color:{x}'
)

,1,2
0,#facade,#639
1,#639,#facade


In [27]:
def show_img(styler, pixel_size=40):
  styler.applymap(
    lambda x: f'color:{x}; background-color:{x}'
  # these get collapsed into just .hide() in pandas 1.5
  ).hide_index(
  ).hide_columns(
  ).set_table_styles([{
    'selector':'td',
    'props':[
      ('font-size','0px'),
      ('width',f'{pixel_size}px'),
      ('height',f'{pixel_size}px')
    ]
  }])
  return styler

show_img(df.style)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#facade,#639
#639,#facade


In [28]:
show_img(df.style, 80)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#facade,#639
#639,#facade


In [29]:
def rgb2hex(r, g, b):
  return f'#{r:02x}{g:02x}{b:02x}'

print(rgb2hex(255, 255, 255))
print(rgb2hex(0,0,0))

#ffffff
#000000


In [30]:
# Do the Mona Lisa
from PIL import Image
mona_lisa = np.array(Image.open('LaGioconda50.bmp').getdata())
mona_lisa = mona_lisa.reshape(71, 50, 3)
print(mona_lisa.shape)
print(len(mona_lisa[0]))
print(mona_lisa[0][:5])

# convert RGB channels to single cells with hex values
hex_mona_lisa = []
for row in mona_lisa:
  new_row = []
  for pixel in row:
    new_row.append(rgb2hex(*pixel))
  hex_mona_lisa.append(new_row)

mona_lisa_df = pd.DataFrame.from_records(hex_mona_lisa)
show_img(mona_lisa_df.style, pixel_size=8)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


(71, 50, 3)
50
[[100 103  77]
 [ 98 105  80]
 [105 108  86]
 [105 109  80]
 [107 113  75]]


In [31]:
def in_circle(x, y):
  return x**2 + y**2 < 60

In [32]:
circle_arr = []

for x in range(-25, 26):
  new_row = []
  for y in range(-25, 26):
    if in_circle(x, y):
      new_row.append('mediumslateblue')
    else:
      new_row.append('#639')
  circle_arr.append(new_row)

circle_df = pd.DataFrame(np.array(circle_arr))

In [33]:
show_img(circle_df.style, 4)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639


In [34]:
# write text?
# that seems like a pain
# draw random boxes with random colors
def random_color():
  hex_colors = '123456789abcdef'
  color = '#' 
  for _ in range(6):
    color += random.choice(hex_colors)
  return color


def add_random_box(df):
  max_height, max_width = df.shape
  # -1 to avoid off-by-1 errors from shape vs .iloc[]
  max_height -= 1
  max_width -= 1
  outer_limit = 10
  
  # Start completely off the plot
  height_off_plot = max_height + outer_limit + 5
  width_off_plot = max_width + outer_limit + 5
  h1 = height_off_plot
  h2 = height_off_plot
  w1 = width_off_plot
  w2 = width_off_plot

  # pick numbers until we get a set that work
  while (h1 > max_height and h2 > max_height) \
        or (w1 > max_width and w2 > max_width) \
        or (h1 < 0 and h2 < 0) \
        or (w1 < 0 and w2 < 0):
    h1 = random.randint(-outer_limit, max_height + outer_limit)
    h2 = random.randint(-outer_limit, max_height + outer_limit)
    w1 = random.randint(-outer_limit, max_width + outer_limit)
    w2 = random.randint(-outer_limit, max_width + outer_limit)

    # reorder them
    if h1 > h2:
      h1, h2 = h2, h1
    if w1 > w2:
      w1, w2 = w2, w1


  color = random_color()
  # draw vertical sides
  try:
    for i in range(max(0, h1), min(max_height, h2)+1):
      # print(i)
      if w1 >= 0:
        df.iloc[i, w1] = color
      # else:
        # print(f'w1 too small: {w1}')
      if w2 <= max_width:
        df.iloc[i, w2] = color
      # else:
        # print(f'w2 too big: {w2}')
  except Exception as e:
    print('\nbroke in first loop\n')
    vars_i_want = deepcopy(vars())
    vars_i_want.pop('df')
    pprint(vars_i_want)
    raise e

  # draw horizontal sides
  try:
    for i in range(max(0, w1), min(max_width, w2)+1):
      # print(i)
      if h1 >= 0:
        df.iloc[h1, i] = color
      # else:
        # print(f'h1 too small: {h1}')
      if h2 <= max_height:
        df.iloc[h2, i] = color
      # else:
        # print(f'h2 too big: {h2}')
  except Exception as e:
    print('\nbroke in second loop\n')
    vars_i_want = deepcopy(vars())
    vars_i_want.pop('df')
    pprint(vars_i_want)
    raise e
  
  return df.copy()

In [35]:
show_img(add_random_box(circle_df).style, 4)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639,#639
#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1,#eac7a1


In [36]:
def create_canvas(height=50, width=50, background_color='#fff'):
  return pd.DataFrame(np.full((height, width), background_color))

c1 = create_canvas(background_color='navy')
show_img(c1.style, 4)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy
navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy,navy


In [37]:
c1 = create_canvas(background_color='#eee')
for _ in range(12):
  add_random_box(c1)
show_img(c1.style, 4)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#eee,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#dd3416,#dd3416,#dd3416,#dd3416,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#ba9d56,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535,#77a535
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee
#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#e2dbe9,#e2dbe9,#eee,#dd3416,#eee,#eee,#eee,#eee,#ba9d56,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee,#eee


In [38]:
def generative_art():
  canvas = create_canvas(background_color=random_color())
  for i in range(random.randint(5, 50)):
    add_random_box(canvas)
  return canvas


In [39]:
show_img(generative_art().style, 4)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#b7438d,#dd36d9,#22d5fc,#b7438d,#22d5fc,#22d5fc,#22d5fc,#dd36d9,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#47ab3b,#26a8c6,#22d5fc,#26a8c6,#4d3c91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc
#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#dd36d9,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#dd36d9,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#47ab3b,#6951d8,#6951d8,#6951d8,#4d3c91,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8,#fbb34d,#6951d8,#6951d8,#6951d8,#6951d8,#6951d8
#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#bc3cff,#b7438d,#dd36d9,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#dd36d9,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#47ab3b,#9bd83e,#9bd83e,#9bd83e,#4d3c91,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#fbb34d,#9bd83e,#9bd83e,#9bd83e,#9bd83e,#6951d8
#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#bc3cff,#b7438d,#dd36d9,#22d5fc,#b7438d,#22d5fc,#bc3cff,#22d5fc,#dd36d9,#22d5fc,#22d5fc,#227a91,#227a91,#227a91,#227a91,#227a91,#227a91,#227a91,#227a91,#227a91,#227a91,#227a91,#47ab3b,#227a91,#227a91,#227a91,#4d3c91,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#6951d8
#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#bc3cff,#b7438d,#dd36d9,#22d5fc,#b7438d,#22d5fc,#bc3cff,#22d5fc,#dd36d9,#22d5fc,#22d5fc,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#47ab3b,#26a8c6,#22d5fc,#26a8c6,#4d3c91,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#6951d8
#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#bc3cff,#b7438d,#dd36d9,#22d5fc,#b7438d,#22d5fc,#bc3cff,#22d5fc,#dd36d9,#22d5fc,#22d5fc,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#47ab3b,#26a8c6,#22d5fc,#26a8c6,#4d3c91,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#6951d8
#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#dd36d9,#11a145,#11a145,#11a145,#11a145,#11a145,#dd36d9,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#11a145,#47ab3b,#26a8c6,#22d5fc,#26a8c6,#4d3c91,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#6951d8
#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#11a145,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#bc3cff,#b7438d,#dd36d9,#22d5fc,#b7438d,#22d5fc,#bc3cff,#22d5fc,#dd36d9,#22d5fc,#22d5fc,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#47ab3b,#26a8c6,#22d5fc,#26a8c6,#4d3c91,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#6951d8
#1978ad,#1978ad,#1978ad,#1978ad,#1978ad,#1978ad,#11a145,#1978ad,#1978ad,#1978ad,#1978ad,#1978ad,#bc3cff,#1978ad,#dd36d9,#1978ad,#b7438d,#22d5fc,#bc3cff,#22d5fc,#dd36d9,#22d5fc,#22d5fc,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#47ab3b,#26a8c6,#22d5fc,#26a8c6,#4d3c91,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#6951d8
#36fa15,#36fa15,#36fa15,#36fa15,#36fa15,#36fa15,#11a145,#36fa15,#36fa15,#36fa15,#36fa15,#22d5fc,#bc3cff,#b7438d,#dd36d9,#1978ad,#b7438d,#22d5fc,#bc3cff,#22d5fc,#dd36d9,#22d5fc,#22d5fc,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#47ab3b,#26a8c6,#22d5fc,#26a8c6,#4d3c91,#227a91,#22d5fc,#22d5fc,#22d5fc,#22d5fc,#fbb34d,#22d5fc,#83df1f,#83df1f,#83df1f,#83df1f


In [40]:
show_img(generative_art().style, 10)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#6f947b,#6f947b,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#abb99e,#3df461,#3df461,#ef5af6,#3df461,#ef5af6,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#3df461,#6f947b,#51e82e,#6f947b,#abb99e,#6f947b
#6f947b,#6f947b,#3df461,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#af7b1a,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#abb99e,#6f947b,#6f947b,#ef5af6,#51e82e,#ef5af6,#6f947b,#6f947b,#e6248f,#6f947b,#41f7fa,#6f947b,#6f947b,#6f947b,#6f947b,#3df461,#6f947b,#51e82e,#6f947b,#abb99e,#6f947b
#6f947b,#6f947b,#3df461,#6f947b,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#2f949c,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#abb99e,#6f947b,#6f947b,#ef5af6,#51e82e,#ef5af6,#6f947b,#6f947b,#e6248f,#6f947b,#41f7fa,#6f947b,#6f947b,#6f947b,#6f947b,#3df461,#6f947b,#51e82e,#6f947b,#abb99e,#6f947b
#6f947b,#6f947b,#3df461,#6f947b,#2f949c,#6f947b,#6f947b,#6f947b,#6f947b,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#abb99e,#51523a,#51523a,#ef5af6,#51523a,#ef5af6,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51523a,#51e82e,#6f947b,#abb99e,#6f947b
#7e77fc,#7e77fc,#3df461,#7e77fc,#2f949c,#7e77fc,#7e77fc,#7e77fc,#7e77fc,#51523a,#7e77fc,#7e77fc,#8c437b,#8c437b,#8c437b,#8c437b,#8c437b,#8c437b,#8c437b,#685771,#685771,#685771,#685771,#685771,#685771,#685771,#685771,#685771,#685771,#abb99e,#685771,#685771,#ef5af6,#685771,#ef5af6,#685771,#685771,#e6248f,#8c437b,#41f7fa,#6f947b,#6f947b,#6f947b,#6f947b,#3df461,#51523a,#51e82e,#6f947b,#abb99e,#6f947b
#6f947b,#6f947b,#3df461,#6f947b,#2f949c,#6f947b,#6f947b,#7d3b49,#7d3b49,#51523a,#7d3b49,#7d3b49,#7d3b49,#7d3b49,#7d3b49,#7d3b49,#7d3b49,#7d3b49,#6f947b,#6f947b,#af7b1a,#2f949c,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#abb99e,#6f947b,#6f947b,#ef5af6,#51e82e,#ef5af6,#6f947b,#6f947b,#e6248f,#8c437b,#41f7fa,#6f947b,#6f947b,#6f947b,#6f947b,#3df461,#51523a,#51e82e,#6f947b,#abb99e,#6f947b
#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#51523a,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#abb99e,#d8b182,#d8b182,#ef5af6,#d8b182,#ef5af6,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#d8b182,#51523a,#d8b182,#d8b182,#abb99e,#d8b182
#6f947b,#6f947b,#3df461,#6f947b,#2f949c,#6f947b,#6f947b,#7d3b49,#6f947b,#51523a,#6f947b,#595f87,#8c437b,#6f947b,#6f947b,#7e77fc,#6f947b,#7d3b49,#6f947b,#6f947b,#af7b1a,#2f949c,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#abb99e,#6f947b,#6f947b,#ef5af6,#51e82e,#ef5af6,#6f947b,#6f947b,#e6248f,#8c437b,#41f7fa,#6f947b,#6f947b,#6f947b,#6f947b,#3df461,#51523a,#51e82e,#6f947b,#abb99e,#6f947b
#6f947b,#6f947b,#3df461,#6f947b,#2f949c,#6f947b,#6f947b,#7d3b49,#6f947b,#51523a,#6f947b,#595f87,#8c437b,#6f947b,#6f947b,#7e77fc,#6f947b,#7d3b49,#6f947b,#6f947b,#af7b1a,#2f949c,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#abb99e,#6f947b,#6f947b,#ef5af6,#51e82e,#ef5af6,#6f947b,#6f947b,#e6248f,#8c437b,#41f7fa,#6f947b,#6f947b,#6f947b,#6f947b,#3df461,#51523a,#51e82e,#6f947b,#abb99e,#6f947b
#6f947b,#6f947b,#3df461,#6f947b,#2f949c,#6f947b,#6f947b,#7d3b49,#6f947b,#51523a,#6f947b,#595f87,#8c437b,#6f947b,#6f947b,#7e77fc,#6f947b,#7d3b49,#6f947b,#6f947b,#af7b1a,#2f949c,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#6f947b,#abb99e,#6f947b,#6f947b,#ef5af6,#51e82e,#ef5af6,#6f947b,#6f947b,#e6248f,#8c437b,#41f7fa,#6f947b,#6f947b,#6f947b,#6f947b,#3df461,#51523a,#51e82e,#6f947b,#abb99e,#6f947b


In [41]:
# what about some % chance to draw
# only 50% chance to overwrite an existing box line
# keep track of seen corners and don't let boxes overlap so they only intersect


In [42]:
def add_random_boxes_intersections(df, num_boxes):
  max_height, max_width = df.shape
  # -1 to avoid off-by-1 errors from shape vs .iloc[]
  max_height -= 1
  max_width -= 1
  outer_limit = 10

  seen_heights = []
  seen_widths = []

  for _ in range(num_boxes):
    # Start completely off the plot
    height_off_plot = max_height + outer_limit + 5
    width_off_plot = max_width + outer_limit + 5
    h1 = height_off_plot
    h2 = height_off_plot
    w1 = width_off_plot
    w2 = width_off_plot

    # pick numbers until we get a set that work
    while (h1 > max_height and h2 > max_height)  \
          or (w1 > max_width and w2 > max_width) \
          or (h1 < 0 and h2 < 0)             \
          or (w1 < 0 and w2 < 0)             \
          or h1 in seen_heights              \
          or h2 in seen_heights              \
          or w1 in seen_widths               \
          or w2 in seen_widths :
      h1 = random.randint(-outer_limit, max_height + outer_limit)
      h2 = random.randint(-outer_limit, max_height + outer_limit)
      w1 = random.randint(-outer_limit, max_width + outer_limit)
      w2 = random.randint(-outer_limit, max_width + outer_limit)

      # reorder them
      if h1 > h2:
        h1, h2 = h2, h1
      if w1 > w2:
        w1, w2 = w2, w1

    seen_heights.extend([h1, h2])
    seen_widths.extend([w1, w2])

    color = random_color()
    # draw vertical sides
    try:
      for i in range(max(0, h1), min(max_height, h2)+1):
        # print(i)
        if w1 >= 0:
          df.iloc[i, w1] = color
        # else:
          # print(f'w1 too small: {w1}')
        if w2 <= max_width:
          df.iloc[i, w2] = color
        # else:
          # print(f'w2 too big: {w2}')
    except Exception as e:
      print('\nbroke in first loop\n')
      vars_i_want = deepcopy(vars())
      vars_i_want.pop('df')
      pprint(vars_i_want)
      raise e

    # draw horizontal sides
    try:
      for i in range(max(0, w1), min(max_width, w2)+1):
        # print(i)
        if h1 >= 0:
          df.iloc[h1, i] = color
        # else:
          # print(f'h1 too small: {h1}')
        if h2 <= max_height:
          df.iloc[h2, i] = color
        # else:
          # print(f'h2 too big: {h2}')
    except Exception as e:
      print('\nbroke in second loop\n')
      vars_i_want = deepcopy(vars())
      vars_i_want.pop('df')
      pprint(vars_i_want)
      raise e
  
  return df.copy()

In [43]:
def generate_intersection_art():
  canvas = create_canvas(background_color=random_color())
  add_random_boxes_intersections(canvas, random.randint(5, 20))
  return canvas

In [44]:
show_img(generate_intersection_art().style, 6)

<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styler.applymap(
<ipython-input-27-fc28f1b10698>:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='columns')`
  styler.applymap(


#38ba16,#24c3d3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#1b5331,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#e37aa2,#38ba16,#38ba16
#38ba16,#24c3d3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#1b5331,#e37aa2,#f65e29,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#1b5331,#e37aa2,#1b5331,#1b5331
#24c3d3,#24c3d3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#f65e29,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16
#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#f65e29,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16
#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#f65e29,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#77a5e3,#77a5e3,#77a5e3,#77a5e3,#77a5e3,#77a5e3,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16
#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#f65e29,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16
#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#f65e29,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16
#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#65a567,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16
#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#65a567,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#e37aa2,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#65a567,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#c472fb,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16
#35aa51,#35aa51,#35aa51,#35aa51,#35aa51,#c472fb,#35aa51,#35aa51,#35aa51,#35aa51,#65a567,#35aa51,#35aa51,#35aa51,#35aa51,#35aa51,#e37aa2,#f65e29,#35aa51,#35aa51,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#65a567,#77a5e3,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#77a5e3,#38ba16,#c472fb,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#38ba16,#e37aa2,#38ba16,#38ba16


In [45]:
# maybe an animation of adding each box

In [46]:
# incorporate Piet somehow?

In [47]:
# a version where every box has at least one corner outside the plot?

In [48]:
# different shapes
from dataclasses import dataclass

def add_random_spirals(df, num_circles):
  @dataclass
  class DirCounter:
    dir: int
    count: int

  for _ in range(num_circles):
    center_x = random.randint(0, df.shape[1]-1)
    center_y = random.randint(0, df.shape[0]-1)
    df.iloc[center_y, center_x] = '#0000ff'

    l = random.randint(7, max(df.shape))
    print(l)
    x = deepcopy(center_x)
    y = deepcopy(center_y)
    # very weird system of counters
    # to go right twice then left twice
    # while going up twice then down twice
    dirx = DirCounter(0,0)
    diry = DirCounter(0,0)
    for _ in range(l):
      # horizontal
      if dirx.dir and dirx.count < 2:
        x += 1
        dirx.count += 1
      elif dirx.dir and dirx.count == 2:
        x -=  1
        dirx.dir = 0
        dirx.count = 1
      elif dirx.count < 2:
        x -= 1
        dirx.count += 1
      else:
        x += 1
        dirx.dir = 1
        dirx.count = 1

      # vertical
      if diry.dir and diry.count < 2:
        y += 1
        diry.count += 1
      elif diry.dir and diry.count == 2:
        y -=  1
        diry.dir = 0
        diry.count = 1
      elif diry.count < 2:
        y -= 1
        diry.count += 1
      else:
        y += 1
        diry.dir = 1
        diry.count = 1
      
      print('x', x)
      print('y', y)
      df.iloc[y,x] = '#0000ff'

        
      

  #return df.copy()